In [2]:
import requests
import numpy as np
import pandas as pd

# Function to fetch data based on distance, latitude, and longitude
def fetch_safecast_data(latitude, longitude, date, interval):
    # URL of the JSON API with dynamic variables
    columns = ['id', 'user_id', 'value', 'unit', 'location_name', 'device_id', 'original_id', 'measurement_import_id', 'captured_at', 'height', 'devicetype_id', 'sensor_id', 'station_id', 'channel_id', 'latitude', 'longitude']
    end_date = date + pd.Timedelta(days=interval)
    hours = np.arange(0, 24, 3)
    batch = pd.DataFrame(columns=columns)
    for i in range(0, len(hours)-2):
        url = f"https://api.safecast.org/en-US/measurements?captured_after={date.year}%2F{date.month}%2F{date.day}+{hours[i]}%3A00%3A00&captured_before={date.year}%2F{date.month}%2F{date.day}+{hours[i+1]}%3A00%3A00&format=json&latitude={latitude}&longitude={longitude}"

        response = requests.get(url)


        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse JSON response
            data = response.json()
            batch = pd.concat([batch, pd.DataFrame(data)], ignore_index=True)
        else:
            print("Failed to fetch data:", response.status_code)
            continue
    avg_batch= batch[['value', 'unit', 'location_name', 'captured_at', 'device_id', 'height', 'devicetype_id', 'station_id', 'latitude', 'longitude']].groupby(['latitude', 'longitude', 'unit']).agg({
    'value': 'mean',
    'location_name': 'first',
    'captured_at' : 'first',
    'device_id': 'first', 
    'height': 'first', 
    'devicetype_id': 'first', 
    'station_id': 'first'
}).reset_index()

    return avg_batch
    
def fetch_all(latitude_range, longitude_range, date_range, interval):
    columns = ['value', 'unit', 'location_name', 'captured_at', 'device_id', 'height', 'devicetype_id', 'station_id', 'latitude', 'longitude']
    result = pd.DataFrame(columns=columns)

    latitude = np.linspace(latitude_range[0], latitude_range[1], interval)
    longitude = np.linspace(longitude_range[0], longitude_range[1], interval)
    dates = pd.date_range(start=date_range[0], end=date_range[0], periods=(date_range[1] - date_range[0]).days)
    
    for date in dates:
        for lat in latitude:
            for long in longitude:
                batch = pd.DataFrame(fetch_safecast_data(lat, long, date, 1))
                result = pd.concat([result, batch], ignore_index=True)
    return result


In [3]:
result = fetch_all([34, 35], [134, 135], [pd.Timestamp('2024-03-20'), pd.Timestamp('2024-03-21')], 2)

C:\Users\lopma\AppData\Local\Temp\ipykernel_25276\1241114567.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  batch = pd.concat([batch, pd.DataFrame(data)], ignore_index=True)
C:\Users\lopma\AppData\Local\Temp\ipykernel_25276\1241114567.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, batch], ignore_index=True)
C:\Users\lopma\AppData\Local\Temp\ipykernel_25276\1241114567.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries

In [5]:
result

,value,unit,location_name,captured_at,device_id,height,devicetype_id,station_id,latitude,longitude
0,19.500000,cpm,None,2024-03-20T03:00:03.000Z,100221,13.0,Pointcast V1,None,31.833193,130.301922
1,19.100000,status,None,2024-03-20T03:00:05.000Z,100229,13.0,"DeviceID:10022,Temperature:19.1,Battery Voltag...",None,31.833193,130.301922
2,14.666667,cpm,"Phoenix,AZ",2024-03-20T03:00:35.000Z,4841,NaN,None,None,33.665600,-112.182700
3,26.000000,cpm,"Earl's House, Johns Creek, GA, USA",2024-03-20T06:00:12.066Z,90,615.0,None,None,34.067301,-84.211610
4,0.172000,usv,"Earl's House, Johns Creek, GA, USA",2024-03-20T06:00:12.449Z,90,615.0,None,None,34.067301,-84.211610
...,...,...,...,...,...,...,...,...,...,...
447,19.000000,cpm,"Bad Pyrmont, DE",2024-03-20T06:00:13.000Z,108,NaN,None,None,51.980700,9.234500
448,17.333333,cpm,"Waterland, NL",2024-03-20T03:00:12.000Z,205,NaN,None,None,52.427600,4.971100
449,17.000000,cpm,"Berlin, DE",2024-03-20T00:00:18.000Z,204,NaN,None,None,52.449400,13.312700
450,10.650000,PM10 ug/m3,None,2024-03-20T06:00:16.385Z,244,NaN,None,None,53.864000,-3.047000


In [6]:
import os

if not os.path.exists('data'):
    os.makedirs('data')
    
result.to_csv('data/safecast.csv', index=False)